# Predict regulatory regions from the DNA sequence using keras

In this notebook we illustrate several variants how to predict regulatory regions (of a toy example) from the DNA sequence.
The reference genome is made up of a concatenation of Oct4 and Mafk binding sites and we shall use all regions on chromosome 'pseudo1' as training
and 'pseudo2' as test chromosomes.

In [1]:
import os

import numpy as np
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

from pkg_resources import resource_filename

from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D

from sklearn.metrics import roc_auc_score

/home/wkopp/anaconda3/envs/jdev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from IPython.display import Image

np.random.seed(1234)

First, we need to specify the output directory in which the results are stored.

In [3]:
os.environ['JANGGU_OUTPUT'] = '/home/wkopp/janggu_examples'

Specify the DNA sequence feature order. Order 1, 2 and 3 correspond to mono-, di- and tri-nucleotide based features (see Tutorial).

In [4]:
order = 3

In [5]:
# load the dataset
# The pseudo genome represents just a concatenation of all sequences
# in sample.fa and sample2.fa. Therefore, the results should be almost
# identically to the models obtained from classify_fasta.py.
REFGENOME = resource_filename('janggu', 'resources/pseudo_genome.fa')
# ROI contains regions spanning positive and negative examples
ROI_TRAIN_FILE = resource_filename('janggu', 'resources/roi_train.bed')
ROI_TEST_FILE = resource_filename('janggu', 'resources/roi_test.bed')
# PEAK_FILE only contains positive examples
PEAK_FILE = resource_filename('janggu', 'resources/scores.bed')

Load the datasets for training and testing

In [6]:
# Training input and labels are purely defined genomic coordinates
DNA = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                   roi=ROI_TRAIN_FILE,
                                   binsize=200,
                                   order=order,
                                   cache=True)

# The ReduceDim wrapper transforms the dataset from a 4D object to a 2D table-like representation
LABELS = ReduceDim(Cover.create_from_bed('peaks', roi=ROI_TRAIN_FILE,
                               bedfiles=PEAK_FILE,
                               binsize=200,
                               resolution=200,
                               cache=True,
                               storage='sparse'))


DNA_TEST = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                        roi=ROI_TEST_FILE,
                                        binsize=200,
                                        order=order)

LABELS_TEST = ReduceDim(Cover.create_from_bed('peaks',
                                    bedfiles=PEAK_FILE,
                                    roi=ROI_TEST_FILE,
                                    binsize=200,
                                    resolution=200,
                                    storage='sparse'))


loading from lazy loader
reload /home/wkopp/janggu_examples/datasets/dna/09dc9f2602f1bea4bf22fd52b8adfa21c7500a31f70c0ef7a506f79a4f92b43a.npz
loading from bed lazy loader
reload /home/wkopp/janggu_examples/datasets/peaks/fd9826cf7fb9cc044a6c1354a14688c1be0f0bd9c593fdba2e9af3284a2be099.npz
loading from lazy loader
loading from bed lazy loader


In [7]:
print('training set:', DNA.shape, LABELS.shape)
print('test set:', DNA_TEST.shape, LABELS_TEST.shape)

training set: (7797, 198, 1, 64) (7797, 1)
test set: (200, 198, 1, 64) (200, 1)


Specify a keras model with compatible dimesions input and output dimensions for the example.

In [8]:
xin = Input((200 - order + 1, 1, pow(4, order)))
layer = DnaConv2D(Conv2D(30, (21, 1),
                         activation='relu'))(xin)
layer = GlobalAveragePooling2D()(layer)
output = Dense(1, activation='sigmoid')(layer)



model = Model(xin, output)

model.compile(optimizer='adadelta', loss='binary_crossentropy',
              metrics=['acc'])
model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 198, 1, 64)        0         
_________________________________________________________________
dna_conv2d_1 (DnaConv2D)     (None, 178, 1, 30)        40350     
_________________________________________________________________
global_average_pooling2d_1 ( (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 40,381
Trainable params: 40,381
Non-trainable params: 0
_________________________________________________________________


In [9]:
hist = model.fit(DNA, LABELS, epochs=100)

print('#' * 40)
print('loss: {}, acc: {}'.format(hist.history['loss'][-1],
                                 hist.history['acc'][-1]))
print('#' * 40)

Epoch 1/100
7797/7797 [==============================] - 5s 594us/step - loss: 0.5611 - acc: 0.7354
Epoch 2/100
7797/7797 [==============================] - 4s 453us/step - loss: 0.4607 - acc: 0.7980
Epoch 3/100
7797/7797 [==============================] - 3s 429us/step - loss: 0.4115 - acc: 0.8222
Epoch 4/100
7797/7797 [==============================] - 3s 409us/step - loss: 0.3785 - acc: 0.8424
Epoch 5/100
7797/7797 [==============================] - 3s 424us/step - loss: 0.3513 - acc: 0.8551
Epoch 6/100
7797/7797 [==============================] - 3s 434us/step - loss: 0.3256 - acc: 0.8657
Epoch 7/100
7797/7797 [==============================] - 3s 400us/step - loss: 0.3037 - acc: 0.8821
Epoch 8/100
7797/7797 [==============================] - 3s 396us/step - loss: 0.2801 - acc: 0.8916
Epoch 9/100
7797/7797 [==============================] - 3s 406us/step - loss: 0.2602 - acc: 0.9006
Epoch 10/100
7797/7797 [==============================] - 3s 408us/step - loss: 0.2412 - acc: 0.9127

7797/7797 [==============================] - 3s 408us/step - loss: 0.0184 - acc: 0.9985
Epoch 83/100
7797/7797 [==============================] - 3s 387us/step - loss: 0.0177 - acc: 0.9986
Epoch 84/100
7797/7797 [==============================] - 3s 412us/step - loss: 0.0172 - acc: 0.9987
Epoch 85/100
7797/7797 [==============================] - 3s 412us/step - loss: 0.0168 - acc: 0.9987
Epoch 86/100
7797/7797 [==============================] - 3s 406us/step - loss: 0.0165 - acc: 0.9988
Epoch 87/100
7797/7797 [==============================] - 3s 413us/step - loss: 0.0160 - acc: 0.9992
Epoch 88/100
7797/7797 [==============================] - 3s 412us/step - loss: 0.0156 - acc: 0.9991
Epoch 89/100
7797/7797 [==============================] - 3s 425us/step - loss: 0.0153 - acc: 0.9990
Epoch 90/100
7797/7797 [==============================] - 4s 449us/step - loss: 0.0149 - acc: 0.9988
Epoch 91/100
7797/7797 [==============================] - 3s 407us/step - loss: 0.0145 - acc: 0.9992
Epo

The predictions may be converted back to Cover object and subsequently exported as bigwig in order to inspect the plausibility of the results in the genome browser.

In [10]:

# convert the prediction to a cover object
pred = model.predict(DNA_TEST)
cov_pred = Cover.create_from_array('BindingProba', pred, LABELS_TEST.gindexer)

# predictions (or feature activities) can finally be exported to bigwig
cov_pred.export_to_bigwig(output_dir=os.environ['JANGGU_OUTPUT'])


In [11]:
print("AUC:", roc_auc_score(LABELS_TEST[:], pred))

AUC: 0.9955999999999999
